In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
%pip install fastapi uvicorn nest_asyncio pyngrok
dbutils.library.restartPython()  


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%env NGROK_TOKEN=<your_ngrok_token>

env: NGROK_TOKEN=2hHNSDAsM6qmOj2cnidTVK10KQv_2PnoAyuiGPP5n2DNujxyM


In [0]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI, Query
import pandas as pd
from pyngrok import ngrok
import os
from fastapi.responses import FileResponse

from logger import log_message

In [0]:
base_path = "file:/Workspace/Users/masa.cirkovic@abo.fi/"
medallion_path = base_path + "medallion/"

In [0]:
nest_asyncio.apply()

# Initialize FastAPI app
app = FastAPI()

# Load Golden Layer Data
golden_path = medallion_path + "gold/golden_data"
golden_df = spark.read.format("delta").load(golden_path).toPandas()


# Root endpoint for confirmation
@app.get("/")
async def root():
    log_message("info", "API", "get /", "Root endpoint accessed")
    return {"message": "API is working successfully!"}

# Endpoint to return golden data
@app.get("/golden_data")
async def get_golden_data(
    date: str = Query(None, description="Filter by date (YYYY-MM-DD)"),
    region: str = Query(None, description="Filter by region"),
    company: str = Query(None, description="Filter by company")
):
    filtered_df = golden_df.copy()

    log_message("info", "API", "get /golden_data", f"Request received: date={date}, region={region}, company={company}")
    
    if date:
        try:
            query_date = pd.to_datetime(date)
            filtered_df = filtered_df[filtered_df["date"] == query_date.date()]
        except Exception as e:
            log_message("error", "API", "get /golden_data", f"Invalid date format: {e}")
            return {"error": f"Invalid date format: {e}"}
    
    if region:
        filtered_df = filtered_df[filtered_df["dealer_region"].str.lower().str.contains(region.lower(), na=False)]
    
    if company:
        filtered_df = filtered_df[filtered_df["company_name"].str.lower().str.contains(company.lower(), na=False)]

    return filtered_df.to_dict(orient='records')

@app.get("/download_golden_data")
async def download_golden_data():
    log_message("info", "API", "get /download_golden_data", "Request received")
    
    csv_path = "/tmp/golden_data.csv"
    golden_df.to_csv(csv_path, index=False)
    return FileResponse(csv_path, media_type='text/csv', filename='golden_data.csv')

# Ngrok Tunnel Setup
ngrok_token = os.getenv("NGROK_TOKEN")
ngrok.set_auth_token(ngrok_token)  
public_url = ngrok.connect(8000).public_url
print(f"Your API is live at: {public_url}")


uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [1235921]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Your API is live at: https://cdd1-4-210-115-137.ngrok-free.app
INFO:     130.232.143.64:0 - "GET / HTTP/1.1" 200 OK
INFO:     130.232.143.64:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     130.232.143.64:0 - "GET /golden_data HTTP/1.1" 200 OK
INFO:     130.232.143.64:0 - "GET /golden_data?company=ford HTTP/1.1" 200 OK
INFO:     130.232.143.64:0 - "GET /golden_data?region=pasco HTTP/1.1" 200 OK
INFO:     130.232.143.64:0 - "GET /golden_data?date=2020 HTTP/1.1" 200 OK
INFO:     130.232.143.64:0 - "GET /golden_data?date=not_a_date HTTP/1.1" 200 OK
INFO:     130.232.143.64:0 - "GET /golden_data?date=2022-06-15 HTTP/1.1" 200 OK
INFO:     130.232.143.64:0 - "GET /download_golden_data HTTP/1.1" 200 OK


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can